# The Movies Database
## Descriptive Analysis

Required imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyspark
%matplotlib inline



Bad key "text.kerning_factor" on line 4 in
C:\Users\20111\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


### Cleaning the data
Loading the cast file and exploring its contents 

In [70]:
cast_df = pd.read_csv(r'./data/credits.csv')
cast_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Files are in JSON format which is not friendly to exploration.  
Parsing the data to get the actors info for each movie in a csv, from the cast column.  
(movie_id, actor_id, name, order, gender)
order: the order in the credits with 0 being the star of the movie    
Although crew are critical to movies quality, the audience rarely targets them when heading to a movie, so I will only take the director form the crew column.

In [91]:
# JSON requires name being in double quotes
# ast handles this pretty well 
import ast

After parsing the data getting the director per movie.

In [92]:
x = (cast_df['crew'][0])
y = ast.literal_eval(x)
for i in y:
    if i['job'] == 'Director':
        print(i['name'])

John Lasseter


Getting (movie_id, actor_id, name, order, gender)

In [117]:
x = (cast_df['cast'][0])
y = ast.literal_eval(x)
for k in y[0]:   
    print(k, y[0][k])


cast_id 14
character Woody (voice)
credit_id 52fe4284c3a36847f8024f95
gender 2
id 31
name Tom Hanks
order 0
profile_path /pQFoyx7rp09CJTAb932F2g8Nlho.jpg


Saving clean data into csv files

In [118]:
actors = [("movie_id", "actor_id", "name", "order", "gender")]
cast = cast_df['cast']
ids = cast_df['id']
N = cast.shape[0]

for i in range(N):
    info_str = cast[i]
    id = ids[i]
    info_dicts = ast.literal_eval(info_str)
    for entry in info_dicts:
        actors.append((id, entry['id'], entry['name'], entry['order'], entry['gender']))
    

In [119]:
directors = [("movie_id", "director_id", "name")]
crew = cast_df['crew']

N = crew.shape[0]

for i in range(N):
    info_str = crew[i]
    info_dicts = ast.literal_eval(info_str)
    for entry in info_dicts:
        if entry['job'] == 'Director':
            directors.append((id, entry['name']))

Comparing number of movies to number of diectors it is apparent that some movies have multiple directors.  
Comparing number of actors per movie comes out to be average of 12 actors per movie.

In [104]:
print(len(directors), N)
print(len(actors)/N)

49048 45476
12.36859002550796


Output the cleaned data 

In [112]:
import csv

In [114]:
# open the file in the write mode
f = open('./dataclean/cast.csv', 'w')

# create the csv writer
writer = csv.writer(f)

N = len(actors)
for i in range(N):
    writer.writerow(actors[i])

In [115]:
# open the file in the write mode
f = open('./dataclean/directors.csv', 'w')

# create the csv writer
writer = csv.writer(f)

N = len(directors)
for i in range(N):
    writer.writerow(directors[i])  

Keywords are indicative to desire of watching a movie, maybe more specific than genres.

In [120]:
kw_df = pd.read_csv(r'./data/keywords.csv')
kw_df.head(1)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [123]:
x = kw_df['keywords'][1]
y = ast.literal_eval(x)
print(y)

[{'id': 10090, 'name': 'board game'}, {'id': 10941, 'name': 'disappearance'}, {'id': 15101, 'name': "based on children's book"}, {'id': 33467, 'name': 'new home'}, {'id': 158086, 'name': 'recluse'}, {'id': 158091, 'name': 'giant insect'}]


In [125]:
N = kw_df.shape[0]
keywords = [("movie_id", "keyword_id", "keyword")]
for i in range(N):
    id = kw_df['id'][i]
    info_str = kw_df['keywords'][i]
    info_dicts = ast.literal_eval(info_str)
    for entry in info_dicts:
        keywords.append((id, entry['id'], entry['name']))

In [126]:
# open the file in the write mode
f = open('./dataclean/keywords.csv', 'w')

# create the csv writer
writer = csv.writer(f)

N = len(keywords)
for i in range(N):
    writer.writerow(keywords[i])  

Cleaning movies metadata file

In [169]:
m_df = pd.read_csv(r'./data/movies_metadata.csv')
m_df.head(1)

C:\Users\20111\AppData\Local\Temp\ipykernel_15492\2480996344.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  m_df = pd.read_csv(r'./data/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


Extracting genres

In [170]:
genres = [("movie_id", "genre_id", "genre")]
N = m_df.shape[0]

for i in range(N):
    id = m_df['id'][i]
    info_str = m_df['genres'][i]
    info_dicts = ast.literal_eval(info_str)

    for entry in info_dicts:
        genres.append((id, entry['id'], entry['name']))
    

In [171]:
# open the file in the write mode
f = open('./dataclean/genres.csv', 'w')

# create the csv writer
writer = csv.writer(f)

N = len(genres)
for i in range(N):
    writer.writerow(genres[i])  

Staying with the movies metadata to extact more info.  


In [172]:
m_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Status column in interesting, as only released movies will be of interest in this analysis

In [173]:
m_df.status.unique()

array(['Released', nan, 'Rumored', 'Post Production', 'In Production',
       'Planned', 'Canceled'], dtype=object)

In [174]:
m_df = m_df[m_df.status == 'Released'].reset_index()


Now having the rows of interest let's determine the columns (attributes) of interest for this analysis  

In [175]:
attributes = ['movie_id', 'title', 'popularity', 'budget', 'revenue', 'vote_average', 'vote_count']
m_df.columns

Index(['index', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

Getting these attributes from the metadata

In [176]:
attributes = ['movie_id', 'title', 'release_date', 'popularity', 'budget', 'revenue', 'vote_average', 'vote_count']
meta_data = [attributes]

N = m_df.shape[0]

for i in range(N):
    id = m_df.id[i]
    title = m_df.original_title[i]
    release_date = m_df.release_date[i]
    pop = m_df.popularity[i]
    budget = m_df.budget[i]
    revenue = m_df.revenue[i]
    vote_avg = m_df.vote_average[i]
    vote_cnt = m_df.vote_count[i]

    meta_data.append([id, title, release_date, pop, budget, revenue, vote_avg, vote_cnt])
    

Saving metadata

In [177]:
# open the file in the write mode
f = open('./dataclean/metadata.csv', 'w')

# create the csv writer
writer = csv.writer(f)

N = len(meta_data)
for i in range(N):
    writer.writerow(meta_data[i])  

### Analysis Phase

Initiating a spark session, spart makes use of multiple CPUs and multiple processess, so I will make some aggreagations on the the whole ratings using Apache Spark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('TMDb').getOrCreate()

In [4]:
df = pd.read_csv(r'./data/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [3]:
df_pyspark = spark.read.csv(r'./data/ratings.csv', header= True, inferSchema= True)
df_pyspark.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    110|   1.0|1425941529|
|     1|    147|   4.5|1425942435|
|     1|    858|   5.0|1425941523|
|     1|   1221|   5.0|1425941546|
|     1|   1246|   5.0|1425941556|
|     1|   1968|   4.0|1425942148|
|     1|   2762|   4.5|1425941300|
|     1|   2918|   5.0|1425941593|
|     1|   2959|   4.0|1425941601|
|     1|   4226|   4.0|1425942228|
|     1|   4878|   5.0|1425941434|
|     1|   5577|   5.0|1425941397|
|     1|  33794|   4.0|1425942005|
|     1|  54503|   3.5|1425941313|
|     1|  58559|   4.0|1425942007|
|     1|  59315|   5.0|1425941502|
|     1|  68358|   5.0|1425941464|
|     1|  69844|   5.0|1425942139|
|     1|  73017|   5.0|1425942699|
|     1|  81834|   5.0|1425942133|
+------+-------+------+----------+
only showing top 20 rows



In [6]:
# timestamp is not needed
df_pyspark = df_pyspark.drop('timestamp')

The size of the data is well over 25 Million ratings 

In [8]:
df_pyspark.count()

26024289

The number of movies is over 45k with users count over 250k

In [11]:
print(df_pyspark.select('movieId').distinct().count())
print(df_pyspark.select('userId').distinct().count())

45115
270896


I will be using PySpark to get insightful statistics that would help in the process of analysis and visulaization of data.  
Gathering the info about how many ratings does every movie has and the average rating of every movie, also the users rate of ratings in value and also quantity.

In [14]:
from pyspark.sql import functions as F

In [16]:
df_pyspark.groupBy('movieId').agg(F.mean('rating'), F.count('rating')).show()

+-------+------------------+-------------+
|movieId|       avg(rating)|count(rating)|
+-------+------------------+-------------+
|   1645| 3.516589990241182|        14346|
|   1591|2.6416020262782967|         6317|
|   3175| 3.586550320670942|        16216|
|   1580|3.5733178489322874|        42193|
|  68135| 3.097457627118644|         2478|
|    471| 3.654817548175482|        12195|
|   1088| 3.239810636881426|        13519|
|   1959|3.6369782971619364|         5990|
|   2122| 2.634513274336283|         2825|
|   2866|3.6019714479945617|         1471|
|  36525|3.4823726916620035|         1787|
|   3918|2.9595715272978578|         1447|
|   3997| 2.077287716405606|         2426|
|   6620| 3.789404132628544|         4162|
|   1238|3.9629796163069546|         3336|
|   2142| 3.038054538054538|         2457|
|   8638|3.9675026123301986|         4785|
|   2366|3.4740872335211956|         8162|
|   1342|2.9637979902087093|         3881|
|   3794| 3.250574712643678|          870|
+-------+--

In [17]:
df_pyspark.groupBy('userId').agg(F.mean('rating'), F.count('rating')).show()

+------+------------------+-------------+
|userId|       avg(rating)|count(rating)|
+------+------------------+-------------+
|   148|3.9651162790697674|           43|
|   463|2.8582375478927204|          261|
|   471|3.8238341968911915|          193|
|   496|               4.1|           10|
|   833| 4.333333333333333|            3|
|  1088| 4.466666666666667|           15|
|  1238|3.5942028985507246|           69|
|  1342|              3.25|            6|
|  1580|1.7916666666666667|           24|
|  1591|               4.0|           24|
|  1645| 2.983783783783784|          185|
|  1829| 3.634920634920635|           63|
|  1959|               3.0|            6|
|  2122|              4.05|           10|
|  2142|               4.2|            5|
|  2366| 3.522222222222222|          270|
|  2659| 3.986899563318777|          229|
|  2866|               3.6|           15|
|  3175| 4.266666666666667|           15|
|  3749|               3.6|            5|
+------+------------------+-------

It is vividly apparent that the users vary in their votes averages and the number of movies that they care to give a rating, also it goes without saying that it is vital for our analysis to know which movies are rated heavily or rated highly or both by users.

In [18]:
# saving the insights gained 
movies_agg = df_pyspark.groupBy('movieId').agg(F.mean('rating'), F.count('rating')).toPandas()
users_agg = df_pyspark.groupBy('userId').agg(F.mean('rating'), F.count('rating')).toPandas()

In [20]:
movies_agg.columns = ['movieId', 'rating_avg', 'rating_count']
users_agg.columns = ['userId', 'rating_avg', 'rating_count']

For easier categorizing of the data will be rounding the average votes

In [22]:
movies_agg.round(2, inplace= True)
users_agg.round(2, inplace= True)

movieId           int32
rating_avg      float64
rating_count      int64
dtype: object

In [23]:
# save to dataclean
movies_agg.to_csv(r'./dataclean/movies_ratings.csv')
movies_agg.to_csv(r'./dataclean/users_ratings.csv')